In [1]:
import numpy as np
import pandas as pd
import nltk

In [4]:
amazon_df = pd.read_csv("E:/NEW/amazon_product.csv")
amazon_df.drop('id',axis=1,inplace=True)
amazon_df.head()

,Title,Description,Category
0,Swissmar Capstore Select Storage Rack for 18-...,Swissmar's capstore select 18 storage unit kee...,Home & Kitchen Kitchen & Dining Kitchen Utens...
1,Gemini200 Delta CV-880 Gold Crown Livery Airc...,Welcome to the exciting world of GeminiJets! O...,Toys & Games Hobbies Models & Model Kits Pre-...
2,Superior Threads 10501-2172 Magnifico Cream P...,"For quilting and embroidery, this product is m...","Arts, Crafts & Sewing Sewing Thread & Floss S..."
3,Fashion Angels Color Rox Hair Chox Kit,Experiment with the haute trend of hair chalki...,Beauty & Personal Care Hair Care Hair Colorin...
4,Union Creative Giant Killing Figure 05: Daisu...,From Union Creative. Turn your display shelf i...,Toys & Games › Action Figures & Statues › Sta...


In [5]:
print(amazon_df.isnull().sum())

Title          0
Description    0
Category       0
dtype: int64


In [6]:
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

def tokenize_stem(text):
    tokens = nltk.word_tokenize(text.lower())
    stemmed = [stemmer.stem(w) for w in tokens]
    return " ".join(stemmed)

amazon_df['stemmed_tokens'] = amazon_df.apply(
    lambda row: tokenize_stem(row['Title'] + " " + row["Description"]),
    axis=1
)
amazon_df.head()

,Title,Description,Category,stemmed_tokens
0,Swissmar Capstore Select Storage Rack for 18-...,Swissmar's capstore select 18 storage unit kee...,Home & Kitchen Kitchen & Dining Kitchen Utens...,swissmar capstor select storag rack for 18-pac...
1,Gemini200 Delta CV-880 Gold Crown Livery Airc...,Welcome to the exciting world of GeminiJets! O...,Toys & Games Hobbies Models & Model Kits Pre-...,gemini200 delta cv-880 gold crown liveri aircr...
2,Superior Threads 10501-2172 Magnifico Cream P...,"For quilting and embroidery, this product is m...","Arts, Crafts & Sewing Sewing Thread & Floss S...",superior thread 10501-2172 magnifico cream puf...
3,Fashion Angels Color Rox Hair Chox Kit,Experiment with the haute trend of hair chalki...,Beauty & Personal Care Hair Care Hair Colorin...,fashion angel color rox hair chox kit experi w...
4,Union Creative Giant Killing Figure 05: Daisu...,From Union Creative. Turn your display shelf i...,Toys & Games › Action Figures & Statues › Sta...,union creativ giant kill figur 05 : daisuk tsu...


In [7]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

stemmer = SnowballStemmer('english')

def tokenize_stem(text):
    tokens = nltk.word_tokenize(text.lower())
    stemmed = [stemmer.stem(w) for w in tokens]
    return " ".join(stemmed)

amazon_df['stemmed_tokens'] = amazon_df.apply(
    lambda row: tokenize_stem(row['Title'] + " " + row["Description"]),
    axis=1
)

tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split())

def cosine_sim(txt1, txt2):
    matrix = tfidf_vectorizer.fit_transform([txt1, txt2])
    return cosine_similarity(matrix[0:1], matrix[1:2])[0][0]

def search_product(query):
    stemmed_query = tokenize_stem(query)
    amazon_df['similarity'] = amazon_df['stemmed_tokens'].apply(lambda x: cosine_sim(stemmed_query, x))
    res = amazon_df.sort_values(by=['similarity'], ascending=False).head(10)[['Title', 'Description', 'Category']]
    return res

search_product('pencil')

D:\anaconda\anaconda1\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,Title,Description,Category
471,Eisen 2-Hole Square Sharpener with Slide-Over...,The Eisen 2-Hole Square Sharpener has a cover ...,Office Products › Office & School Supplies › ...
258,"Galt Toys, First Sewing","Make a butterfly bag, Neck purse, picture fram...",Toys & Games › Arts & Crafts › Craft Kits › S...
575,"Milani Runway Eyes Fashion Eyeshadow, Backsta...","Double take lip color - surgar and spice, a do...",Beauty & Personal Care › Makeup › Eyes › Eyes...
62,Raymond Geddes Dual Hole Smart Phone Sharpene...,"With colorful app icons, this detailed Pencil ...",Office Products › Office & School Supplies › ...
505,"Mavala Eye-Lite Khol Kajal Pencil Eyeliner, G...",It is allows to ring the eyes for a light clas...,Beauty & Personal Care Makeup Eyes Eyeliner
265,"Rikki Knight Letter""T"" Blue Houndstooth Monog...","Whether for the home or the office, the Letter...",Office Products › Office & School Supplies › ...
268,LeapFrog LeapReader Writing Workbook: Learn t...,Build early math skills with a trip to the Doo...,Toys & Games Kids' Electronics Electronic Lea...
461,Rikki Knight Claude Monet Jardin à Sainte-Adr...,With the four exciting colors that Samsung has...,"Cell Phones & Accessories Cases, Holsters & S..."
0,Swissmar Capstore Select Storage Rack for 18-...,Swissmar's capstore select 18 storage unit kee...,Home & Kitchen Kitchen & Dining Kitchen Utens...
440,"HUGGIES OverNites Diapers, Size 3 (16-28 lb.)...","The most absorbent HUGGIES night time diapers,...",Baby Products Diapering Disposable Diapers
